In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib widget
import numpy as np
import pandas as pd
import torch
import torchvision
from torch import optim
import matplotlib.pyplot as plt
import torch.nn.functional as F
from sklearn.metrics import pairwise_distances
from torch.optim.lr_scheduler import ReduceLROnPlateau
from tqdm.notebook import tqdm
import os
import sys
import time
import torch.nn as nn
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from stream_generators.mit_bih import MITBIH
from utils.MatplotlibUtils import reduce_dims_and_plot
from network.auto_encoder import AutoEncoder
from losses.knn_loss import KNNLoss, ClassificationKNNLoss
from sklearn.cluster import DBSCAN
from sklearn.mixture import GaussianMixture
from soft_decision_tree.sdt_model import SDT
from sklearn.metrics import davies_bouldin_score

In [2]:
k = 32
tree_depth = 10
batch_size = 512
device = 'cuda'
train_data_path = r'/mnt/qnap/ekosman/mitbih_train.csv'
test_data_path = r'/mnt/qnap/ekosman/mitbih_test.csv'

In [3]:
train_data_iter = torch.utils.data.DataLoader(MITBIH(train_data_path),
                                             batch_size=batch_size,
                                             shuffle=True,
                                             num_workers=1,
                                             pin_memory=True,
                                             drop_last=True)

test_data_iter = torch.utils.data.DataLoader(MITBIH(test_data_path),
                                             batch_size=batch_size,
                                             shuffle=True,
                                             num_workers=1,
                                             pin_memory=True)

In [4]:
class ConvBlock(nn.Module):
    def __init__(self):
        super(ConvBlock, self).__init__()
        self.conv1 = nn.Conv1d(32, 32, kernel_size=5, stride=1, padding=2)
        self.conv2 = nn.Conv1d(32, 32, kernel_size=5, stride=1, padding=2)
        self.relu1 = nn.ReLU()
        self.relu2 = nn.ReLU()
        self.pool = nn.MaxPool1d(kernel_size=5, stride=2)
        
    def forward(self, x):
        y = x
        y = self.conv1(y)
        y = self.relu1(y)
        y = self.conv2(y)
        y = y + x
        y = self.relu2(y)
        y = self.pool(y)
        return y


class ECGModel(nn.Module):
    def __init__(self):
        super(ECGModel, self).__init__()
        self.conv = nn.Conv1d(1, 32, kernel_size=5, stride=1, padding=1)
        self.block1 = ConvBlock()
        self.block2 = ConvBlock()
        self.block3 = ConvBlock()
        self.block4 = ConvBlock()
        self.block5 = ConvBlock()
        self.fc1 = nn.Linear(64, 32)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(32, 5)

    def forward(self, x, return_interm=False):
        x = self.conv(x)
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.block4(x)
        x = self.block5(x)
        interm = x.flatten(1)
        x = self.fc1(interm)
        x = self.relu(x)
        x = self.fc2(x)
        
        if return_interm:
            return x, interm
        
        return x

In [5]:
knn_crt = ClassificationKNNLoss(k=k).to(device)

def train(model, loader, optimizer, device):
    model.train()

    total_loss = 0
    for iteration, (batch, target) in enumerate(loader):
        batch = batch.to(device)
        target = target.to(device)
        outputs, interm = model(batch, return_interm=True)
        mse_loss = F.cross_entropy(outputs, target)
        mse_loss = mse_loss.sum(dim=-1).mean()
        try:
            knn_loss = knn_crt(interm, target)
            if torch.isinf(knn_loss):
                knn_loss = torch.tensor(0).to(device)
        except ValueError:
            knn_loss = torch.tensor(0).to(device)
        loss = mse_loss + knn_loss
        total_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if iteration % log_every == 0:
            print(f"Epoch {epoch} / {epochs} | iteration {iteration} / {len(loader)} | Total Loss: {loss.item()} | KNN Loss: {knn_loss.item()} | CLS Loss: {mse_loss.item()}")

    return total_loss / len(loader)

@torch.no_grad()
def test(model, loader, device):
    model.eval()
    
    correct = 0
    for iteration, (batch, target) in enumerate(loader):
        batch = batch.to(device)
        target = target.to(device)
        y_pred = model(batch).argmax(dim=-1)
        correct += y_pred.eq(target.view(-1).data).sum()
    
    return correct / len(loader.dataset)

In [6]:
epochs = 200
lr = 1e-3
log_every = 10

model = ECGModel().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
num_params = sum([p.numel() for p in model.parameters()])
print(f'#Params: {num_params}')

#Params: 53957


In [7]:
best_valid_acc = 0
losses = []
train_accs = []
val_accs = []
for epoch in range(1, epochs + 1):
    loss = train(model, train_data_iter, optimizer, device)
#     print(f"Loss: {loss} =============================")
    losses.append(loss)
    train_acc = test(model, train_data_iter, device)
    train_accs.append(train_acc)
    valid_acc = test(model, test_data_iter, device)
    val_accs.append(valid_acc)
    if valid_acc > best_valid_acc:
        best_valid_acc = valid_acc
    if epoch % 1 == 0:
        print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, '
              f'Train: {train_acc:.4f}, Valid: {valid_acc:.4f}, '
              f'Best: {best_valid_acc:.4f}')

Epoch 1 / 200 | iteration 0 / 171 | Total Loss: 7.358572006225586 | KNN Loss: 5.69699239730835 | CLS Loss: 1.6615793704986572
Epoch 1 / 200 | iteration 10 / 171 | Total Loss: 4.858458518981934 | KNN Loss: 4.118841648101807 | CLS Loss: 0.739617109298706
Epoch 1 / 200 | iteration 20 / 171 | Total Loss: 4.7575297355651855 | KNN Loss: 4.001819610595703 | CLS Loss: 0.7557100653648376
Epoch 1 / 200 | iteration 30 / 171 | Total Loss: 4.550957679748535 | KNN Loss: 3.9674794673919678 | CLS Loss: 0.5834780335426331
Epoch 1 / 200 | iteration 40 / 171 | Total Loss: 4.4556169509887695 | KNN Loss: 3.8908891677856445 | CLS Loss: 0.5647277235984802
Epoch 1 / 200 | iteration 50 / 171 | Total Loss: 4.35487699508667 | KNN Loss: 3.8714163303375244 | CLS Loss: 0.48346057534217834
Epoch 1 / 200 | iteration 60 / 171 | Total Loss: 4.348923206329346 | KNN Loss: 3.8277058601379395 | CLS Loss: 0.5212175250053406
Epoch 1 / 200 | iteration 70 / 171 | Total Loss: 4.310129642486572 | KNN Loss: 3.895495653152466 | CL

Epoch 4 / 200 | iteration 80 / 171 | Total Loss: 3.8621749877929688 | KNN Loss: 3.7199788093566895 | CLS Loss: 0.14219610393047333
Epoch 4 / 200 | iteration 90 / 171 | Total Loss: 3.846350908279419 | KNN Loss: 3.7377986907958984 | CLS Loss: 0.10855214297771454
Epoch 4 / 200 | iteration 100 / 171 | Total Loss: 3.8782801628112793 | KNN Loss: 3.679821252822876 | CLS Loss: 0.19845889508724213
Epoch 4 / 200 | iteration 110 / 171 | Total Loss: 3.8147568702697754 | KNN Loss: 3.693053722381592 | CLS Loss: 0.121703140437603
Epoch 4 / 200 | iteration 120 / 171 | Total Loss: 3.862760305404663 | KNN Loss: 3.7014079093933105 | CLS Loss: 0.16135230660438538
Epoch 4 / 200 | iteration 130 / 171 | Total Loss: 3.8173587322235107 | KNN Loss: 3.670098304748535 | CLS Loss: 0.14726035296916962
Epoch 4 / 200 | iteration 140 / 171 | Total Loss: 3.7751920223236084 | KNN Loss: 3.7059693336486816 | CLS Loss: 0.06922276318073273
Epoch 4 / 200 | iteration 150 / 171 | Total Loss: 3.875886917114258 | KNN Loss: 3.701

Epoch 7 / 200 | iteration 160 / 171 | Total Loss: 3.7856101989746094 | KNN Loss: 3.687330722808838 | CLS Loss: 0.09827938675880432
Epoch 7 / 200 | iteration 170 / 171 | Total Loss: 3.7485692501068115 | KNN Loss: 3.6494884490966797 | CLS Loss: 0.09908083826303482
Epoch: 007, Loss: 3.7624, Train: 0.9775, Valid: 0.9732, Best: 0.9732
Epoch 8 / 200 | iteration 0 / 171 | Total Loss: 3.782048463821411 | KNN Loss: 3.664184093475342 | CLS Loss: 0.11786431819200516
Epoch 8 / 200 | iteration 10 / 171 | Total Loss: 3.771681070327759 | KNN Loss: 3.6252777576446533 | CLS Loss: 0.1464032530784607
Epoch 8 / 200 | iteration 20 / 171 | Total Loss: 3.708716869354248 | KNN Loss: 3.6196749210357666 | CLS Loss: 0.0890420526266098
Epoch 8 / 200 | iteration 30 / 171 | Total Loss: 3.746598958969116 | KNN Loss: 3.6318962574005127 | CLS Loss: 0.11470276117324829
Epoch 8 / 200 | iteration 40 / 171 | Total Loss: 3.7737317085266113 | KNN Loss: 3.6878867149353027 | CLS Loss: 0.08584505319595337
Epoch 8 / 200 | itera

Epoch 11 / 200 | iteration 50 / 171 | Total Loss: 3.7452592849731445 | KNN Loss: 3.6759467124938965 | CLS Loss: 0.06931264698505402
Epoch 11 / 200 | iteration 60 / 171 | Total Loss: 3.735203742980957 | KNN Loss: 3.6836297512054443 | CLS Loss: 0.05157403647899628
Epoch 11 / 200 | iteration 70 / 171 | Total Loss: 3.7291078567504883 | KNN Loss: 3.664486885070801 | CLS Loss: 0.06462091207504272
Epoch 11 / 200 | iteration 80 / 171 | Total Loss: 3.723984479904175 | KNN Loss: 3.665686845779419 | CLS Loss: 0.05829761177301407
Epoch 11 / 200 | iteration 90 / 171 | Total Loss: 3.703261613845825 | KNN Loss: 3.6206865310668945 | CLS Loss: 0.0825750008225441
Epoch 11 / 200 | iteration 100 / 171 | Total Loss: 3.743879795074463 | KNN Loss: 3.671294927597046 | CLS Loss: 0.07258490473031998
Epoch 11 / 200 | iteration 110 / 171 | Total Loss: 3.729762315750122 | KNN Loss: 3.6473186016082764 | CLS Loss: 0.08244366198778152
Epoch 11 / 200 | iteration 120 / 171 | Total Loss: 3.7208974361419678 | KNN Loss: 3

Epoch 14 / 200 | iteration 120 / 171 | Total Loss: 3.726778507232666 | KNN Loss: 3.677480459213257 | CLS Loss: 0.049298059195280075
Epoch 14 / 200 | iteration 130 / 171 | Total Loss: 3.713148832321167 | KNN Loss: 3.667767286300659 | CLS Loss: 0.04538147523999214
Epoch 14 / 200 | iteration 140 / 171 | Total Loss: 3.6706933975219727 | KNN Loss: 3.615631341934204 | CLS Loss: 0.055062130093574524
Epoch 14 / 200 | iteration 150 / 171 | Total Loss: 3.7304608821868896 | KNN Loss: 3.6823830604553223 | CLS Loss: 0.04807782918214798
Epoch 14 / 200 | iteration 160 / 171 | Total Loss: 3.6848113536834717 | KNN Loss: 3.6287119388580322 | CLS Loss: 0.05609948933124542
Epoch 14 / 200 | iteration 170 / 171 | Total Loss: 3.6925601959228516 | KNN Loss: 3.6284523010253906 | CLS Loss: 0.0641079694032669
Epoch: 014, Loss: 3.7151, Train: 0.9834, Valid: 0.9782, Best: 0.9801
Epoch 15 / 200 | iteration 0 / 171 | Total Loss: 3.7169532775878906 | KNN Loss: 3.634455442428589 | CLS Loss: 0.08249791711568832
Epoch 1

...

Epoch 185 / 200 | iteration 150 / 171 | Total Loss: 3.5888113975524902 | KNN Loss: 3.582388401031494 | CLS Loss: 0.006422976031899452
Epoch 185 / 200 | iteration 160 / 171 | Total Loss: 3.605147361755371 | KNN Loss: 3.5967044830322266 | CLS Loss: 0.008442973718047142
Epoch 185 / 200 | iteration 170 / 171 | Total Loss: 3.585801124572754 | KNN Loss: 3.5842857360839844 | CLS Loss: 0.0015154159627854824
Epoch: 185, Loss: 3.6157, Train: 0.9977, Valid: 0.9864, Best: 0.9873
Epoch 186 / 200 | iteration 0 / 171 | Total Loss: 3.6137428283691406 | KNN Loss: 3.60479736328125 | CLS Loss: 0.008945409208536148
Epoch 186 / 200 | iteration 10 / 171 | Total Loss: 3.654035806655884 | KNN Loss: 3.6368348598480225 | CLS Loss: 0.017201025038957596
Epoch 186 / 200 | iteration 20 / 171 | Total Loss: 3.613801956176758 | KNN Loss: 3.6103832721710205 | CLS Loss: 0.003418781328946352
Epoch 186 / 200 | iteration 30 / 171 | Total Loss: 3.6219284534454346 | KNN Loss: 3.596198797225952 | CLS Loss: 0.02572975307703018

Epoch 189 / 200 | iteration 30 / 171 | Total Loss: 3.645723342895508 | KNN Loss: 3.6447136402130127 | CLS Loss: 0.001009657047688961
Epoch 189 / 200 | iteration 40 / 171 | Total Loss: 3.6180782318115234 | KNN Loss: 3.6140189170837402 | CLS Loss: 0.004059290513396263
Epoch 189 / 200 | iteration 50 / 171 | Total Loss: 3.618532180786133 | KNN Loss: 3.59781813621521 | CLS Loss: 0.020713994279503822
Epoch 189 / 200 | iteration 60 / 171 | Total Loss: 3.591970682144165 | KNN Loss: 3.588420867919922 | CLS Loss: 0.0035498125944286585
Epoch 189 / 200 | iteration 70 / 171 | Total Loss: 3.639238119125366 | KNN Loss: 3.6237573623657227 | CLS Loss: 0.015480652451515198
Epoch 189 / 200 | iteration 80 / 171 | Total Loss: 3.6415834426879883 | KNN Loss: 3.6260440349578857 | CLS Loss: 0.015539480373263359
Epoch 189 / 200 | iteration 90 / 171 | Total Loss: 3.607095718383789 | KNN Loss: 3.5986931324005127 | CLS Loss: 0.008402650244534016
Epoch 189 / 200 | iteration 100 / 171 | Total Loss: 3.620838642120361

Epoch 192 / 200 | iteration 90 / 171 | Total Loss: 3.6029415130615234 | KNN Loss: 3.5985329151153564 | CLS Loss: 0.004408539272844791
Epoch 192 / 200 | iteration 100 / 171 | Total Loss: 3.6114110946655273 | KNN Loss: 3.6111257076263428 | CLS Loss: 0.00028540604398585856
Epoch 192 / 200 | iteration 110 / 171 | Total Loss: 3.574134111404419 | KNN Loss: 3.5719404220581055 | CLS Loss: 0.0021937794517725706
Epoch 192 / 200 | iteration 120 / 171 | Total Loss: 3.6275594234466553 | KNN Loss: 3.5978901386260986 | CLS Loss: 0.02966918982565403
Epoch 192 / 200 | iteration 130 / 171 | Total Loss: 3.613825559616089 | KNN Loss: 3.602100372314453 | CLS Loss: 0.011725218035280704
Epoch 192 / 200 | iteration 140 / 171 | Total Loss: 3.6067306995391846 | KNN Loss: 3.604588270187378 | CLS Loss: 0.002142466139048338
Epoch 192 / 200 | iteration 150 / 171 | Total Loss: 3.6595616340637207 | KNN Loss: 3.6351382732391357 | CLS Loss: 0.02442331612110138
Epoch 192 / 200 | iteration 160 / 171 | Total Loss: 3.65298

Epoch 195 / 200 | iteration 150 / 171 | Total Loss: 3.6119351387023926 | KNN Loss: 3.6010773181915283 | CLS Loss: 0.010857892222702503
Epoch 195 / 200 | iteration 160 / 171 | Total Loss: 3.5734660625457764 | KNN Loss: 3.569589614868164 | CLS Loss: 0.0038764362689107656
Epoch 195 / 200 | iteration 170 / 171 | Total Loss: 3.6822028160095215 | KNN Loss: 3.662424325942993 | CLS Loss: 0.01977851800620556
Epoch: 195, Loss: 3.6062, Train: 0.9974, Valid: 0.9858, Best: 0.9873
Epoch 196 / 200 | iteration 0 / 171 | Total Loss: 3.5784912109375 | KNN Loss: 3.5717251300811768 | CLS Loss: 0.006766071543097496
Epoch 196 / 200 | iteration 10 / 171 | Total Loss: 3.6340038776397705 | KNN Loss: 3.611436367034912 | CLS Loss: 0.02256753109395504
Epoch 196 / 200 | iteration 20 / 171 | Total Loss: 3.608452558517456 | KNN Loss: 3.593545436859131 | CLS Loss: 0.01490722969174385
Epoch 196 / 200 | iteration 30 / 171 | Total Loss: 3.632634162902832 | KNN Loss: 3.615565299987793 | CLS Loss: 0.01706889271736145
Epoc

Epoch 199 / 200 | iteration 30 / 171 | Total Loss: 3.609245538711548 | KNN Loss: 3.6047208309173584 | CLS Loss: 0.0045247189700603485
Epoch 199 / 200 | iteration 40 / 171 | Total Loss: 3.595888614654541 | KNN Loss: 3.5880517959594727 | CLS Loss: 0.007836858741939068
Epoch 199 / 200 | iteration 50 / 171 | Total Loss: 3.6148111820220947 | KNN Loss: 3.6065025329589844 | CLS Loss: 0.008308730088174343
Epoch 199 / 200 | iteration 60 / 171 | Total Loss: 3.579241991043091 | KNN Loss: 3.5780508518218994 | CLS Loss: 0.0011911524925380945
Epoch 199 / 200 | iteration 70 / 171 | Total Loss: 3.600745439529419 | KNN Loss: 3.575310468673706 | CLS Loss: 0.02543490007519722
Epoch 199 / 200 | iteration 80 / 171 | Total Loss: 3.6279661655426025 | KNN Loss: 3.617826223373413 | CLS Loss: 0.010140030644834042
Epoch 199 / 200 | iteration 90 / 171 | Total Loss: 3.5906684398651123 | KNN Loss: 3.584307909011841 | CLS Loss: 0.006360555998980999
Epoch 199 / 200 | iteration 100 / 171 | Total Loss: 3.62377357482910

In [8]:
test(model, test_data_iter, device)

tensor(0.9854, device='cuda:0')

In [9]:
plt.figure()
plt.plot(losses, label='train loss')
plt.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
plt.figure()
plt.plot(train_accs, label='train accuracy')
plt.plot(val_accs, label='validation accuracy')
plt.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
test_samples = torch.tensor([])
projections = torch.tensor([])
labels = torch.tensor([])

with torch.no_grad():
    for x, y in tqdm(test_data_iter):
        test_samples = torch.cat([test_samples, x])
        labels = torch.cat([labels, y])
        x = x.to(device)
        _, interm = model(x, True)
        projections = torch.cat([projections, interm.detach().cpu().flatten(1)])

  0%|          | 0/43 [00:00<?, ?it/s]

In [12]:
distances = pairwise_distances(projections)
# distances = np.triu(distances)
distances_f = distances.flatten()

plt.matshow(distances)
plt.colorbar()
plt.figure()
plt.hist(distances_f[distances_f > 0], bins=1000)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [13]:
clusters = DBSCAN(eps=2, min_samples=10).fit_predict(projections)

In [14]:
print(f"Number of inliers: {sum(clusters != -1) / len(clusters)}")

Number of inliers: 0.8574756749349047


In [15]:
perplexity = 100
p = reduce_dims_and_plot(projections[clusters != -1],
                         y=clusters[clusters != -1],
                         title=f'perplexity: {perplexity}',
                         file_name=None,
                         perplexity=perplexity,
                         library='Multicore-TSNE',
                         perform_PCA=False,
                         projected=None,
                         figure_type='2d',
                         show_figure=True,
                         close_figure=False,
                         text=None)    

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Train a Soft-Decision-Tree given the self-labels

## Prepare the dataset

In [16]:
tree_dataset = list(zip(test_samples.flatten(1)[clusters!=-1], clusters[clusters != -1]))
batch_size = 512
tree_loader = torch.utils.data.DataLoader(tree_dataset, batch_size=batch_size, shuffle=True)

# Define how we prune the weights of a node

In [17]:
def prune_node(node_weights, factor=1):
    w = node_weights.cpu().detach().numpy()
    mean_ = np.mean(w)
    std_ = np.std(w)
    node_weights[((mean_ - std_ * factor) < node_weights) & (node_weights < (mean_ + std_ * factor))] = 0
    return node_weights

def prune_node_keep(node_weights, keep=4):
    w = node_weights.cpu().detach().numpy()
    throw_idx = np.argsort(abs(w))[:-keep]
    node_weights[throw_idx] = 0
    return node_weights

def prune_tree(tree_, factor):
    new_weights = tree_.inner_nodes.weight.clone()
    for i in range(new_weights.shape[0]):
        res = prune_node_keep(new_weights[i, :], factor)
        new_weights[i, :] = res

    with torch.no_grad():
        tree_.inner_nodes.weight.copy_(new_weights)
        
def sparseness(x):
    s = []
    for i in range(x.shape[0]):
        x_ = x[i, :]
        sp = (len(x_) - torch.norm(x_, 0).item()) / len(x_)
        s.append(sp)
    return np.mean(s)

def compute_regularization_by_level(tree):
    total_reg = 0
    for i in range(tree.inner_nodes.weight.shape[0]):
        cur_level = np.floor(np.log2(i+1))
        node_reg = torch.norm(tree.inner_nodes.weight[i].view(-1), 2)
        total_reg += 2**(-cur_level) * node_reg
    return total_reg

def show_sparseness(tree):
    avg_sp = sparseness(tree.inner_nodes.weight)
    print(f"Average sparseness: {avg_sp}")
    layer = 0
    sps = []
    for i in range(tree.inner_nodes.weight.shape[0]):
        cur_layer = int(np.floor(np.log2(i+1)))
        if cur_layer != layer:
            print(f"layer {layer}: {np.mean(sps)}")
            sps = []
            layer = cur_layer

        x_ = tree.inner_nodes.weight[i, :]
        sp = (len(x_) - torch.norm(x_, 0).item()) / len(x_)
        sps.append(sp)
        
    return avg_sp

## Training configurations

In [18]:
def do_epoch(model, loader, device, log_interval, losses, accs, epoch, iteration):
    model = model.train()
    for batch_idx, (data, target) in enumerate(loader):
        iteration += 1
        data, target = data.to(device), target.to(device)

        output, penalty = tree.forward(data)

        # Loss
        loss_tree = criterion(output, target.view(-1))

        # Penalty
        loss_tree += penalty

        # Sparse regularization
#         fc_params = torch.cat([x.view(-1) for x in tree.inner_nodes.parameters()])
#         regularization = sparsity_lamda * torch.norm(fc_params, 2)
        regularization = sparsity_lamda * compute_regularization_by_level(tree)
        loss = loss_tree

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        losses.append(loss.item())

        pred = output.data.max(1)[1]
        correct = pred.eq(target.view(-1).data).sum()
        accs.append(correct.item() / data.size()[0])

        # Print training status
        if batch_idx % log_interval == 0:
            print(f"Epoch: {epoch:02d} | Batch: {batch_idx:03d} / {len(loader):03d} | Total loss: {loss.item():.3f} | Reg loss: {regularization.item():.3f} | Tree loss: {loss_tree.item():.3f} | Accuracy: {correct.item() / data.size()[0]:03f} | {round((time.time() - start_time) / iteration, 3)} sec/iter")
            
    return iteration


In [19]:
lr = 5e-3
weight_decay = 5e-4
sparsity_lamda = 2e-3
epochs = 400
log_interval = 100
use_cuda = device != 'cpu'

In [20]:
tree = SDT(input_dim=test_samples.shape[2], output_dim=len(set(clusters)) - 1, depth=tree_depth, lamda=1e-3, use_cuda=use_cuda)
optimizer = torch.optim.Adam(tree.parameters(),
                                 lr=lr,
                                 weight_decay=weight_decay)
criterion = nn.CrossEntropyLoss()
tree = tree.to(device)

In [21]:
losses = []
accs = []
sparsity = []

In [22]:
start_time = time.time()
iteration = 0
for epoch in range(epochs):
    # Training
    avg_sp = show_sparseness(tree)
    sparsity.append(avg_sp)
    iteration = do_epoch(tree, tree_loader, device, log_interval, losses, accs, epoch, iteration)
    
    if epoch % 1 == 0:
        prune_tree(tree, factor=3)
        

Average sparseness: 0.0
layer 0: 0.0
layer 1: 0.0
layer 2: 0.0
layer 3: 0.0
layer 4: 0.0
layer 5: 0.0
layer 6: 0.0
layer 7: 0.0
layer 8: 0.0
Epoch: 00 | Batch: 000 / 037 | Total loss: 2.232 | Reg loss: 0.012 | Tree loss: 2.232 | Accuracy: 0.064453 | 1.322 sec/iter
Average sparseness: 0.9840425531914894
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
layer 5: 0.9840425531914894
layer 6: 0.9840425531914895
layer 7: 0.9840425531914895
layer 8: 0.9840425531914895
Epoch: 01 | Batch: 000 / 037 | Total loss: 2.209 | Reg loss: 0.005 | Tree loss: 2.209 | Accuracy: 0.230469 | 0.829 sec/iter
Average sparseness: 0.9840425531914894
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
layer 5: 0.9840425531914894
layer 6: 0.9840425531914895
layer 7: 0.9840425531914895
layer 8: 0.9840425531914895
Epoch: 02 | Batch: 000 / 037 

Epoch: 20 | Batch: 000 / 037 | Total loss: 1.992 | Reg loss: 0.024 | Tree loss: 1.992 | Accuracy: 0.458984 | 0.819 sec/iter
Average sparseness: 0.9840425531914894
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
layer 5: 0.9840425531914894
layer 6: 0.9840425531914895
layer 7: 0.9840425531914895
layer 8: 0.9840425531914895
Epoch: 21 | Batch: 000 / 037 | Total loss: 1.990 | Reg loss: 0.025 | Tree loss: 1.990 | Accuracy: 0.449219 | 0.819 sec/iter
Average sparseness: 0.9840425531914894
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
layer 5: 0.9840425531914894
layer 6: 0.9840425531914895
layer 7: 0.9840425531914895
layer 8: 0.9840425531914895
Epoch: 22 | Batch: 000 / 037 | Total loss: 1.980 | Reg loss: 0.025 | Tree loss: 1.980 | Accuracy: 0.478516 | 0.818 sec/iter
Average sparseness: 0.9840425531914894
layer 0

Epoch: 40 | Batch: 000 / 037 | Total loss: 1.805 | Reg loss: 0.031 | Tree loss: 1.805 | Accuracy: 0.488281 | 0.816 sec/iter
Average sparseness: 0.9840425531914894
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
layer 5: 0.9840425531914894
layer 6: 0.9840425531914895
layer 7: 0.9840425531914895
layer 8: 0.9840425531914895
Epoch: 41 | Batch: 000 / 037 | Total loss: 1.828 | Reg loss: 0.032 | Tree loss: 1.828 | Accuracy: 0.429688 | 0.816 sec/iter
Average sparseness: 0.9840425531914894
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
layer 5: 0.9840425531914894
layer 6: 0.9840425531914895
layer 7: 0.9840425531914895
layer 8: 0.9840425531914895
Epoch: 42 | Batch: 000 / 037 | Total loss: 1.782 | Reg loss: 0.032 | Tree loss: 1.782 | Accuracy: 0.500000 | 0.815 sec/iter
Average sparseness: 0.9840425531914894
layer 0

Epoch: 60 | Batch: 000 / 037 | Total loss: 1.706 | Reg loss: 0.035 | Tree loss: 1.706 | Accuracy: 0.486328 | 0.804 sec/iter
Average sparseness: 0.9840425531914894
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
layer 5: 0.9840425531914894
layer 6: 0.9840425531914895
layer 7: 0.9840425531914895
layer 8: 0.9840425531914895
Epoch: 61 | Batch: 000 / 037 | Total loss: 1.706 | Reg loss: 0.035 | Tree loss: 1.706 | Accuracy: 0.498047 | 0.804 sec/iter
Average sparseness: 0.9840425531914894
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
layer 5: 0.9840425531914894
layer 6: 0.9840425531914895
layer 7: 0.9840425531914895
layer 8: 0.9840425531914895
Epoch: 62 | Batch: 000 / 037 | Total loss: 1.685 | Reg loss: 0.035 | Tree loss: 1.685 | Accuracy: 0.492188 | 0.803 sec/iter
Average sparseness: 0.9840425531914894
layer 0

Epoch: 80 | Batch: 000 / 037 | Total loss: 1.665 | Reg loss: 0.037 | Tree loss: 1.665 | Accuracy: 0.496094 | 0.795 sec/iter
Average sparseness: 0.9840425531914894
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
layer 5: 0.9840425531914894
layer 6: 0.9840425531914895
layer 7: 0.9840425531914895
layer 8: 0.9840425531914895
Epoch: 81 | Batch: 000 / 037 | Total loss: 1.657 | Reg loss: 0.037 | Tree loss: 1.657 | Accuracy: 0.478516 | 0.794 sec/iter
Average sparseness: 0.9840425531914894
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
layer 5: 0.9840425531914894
layer 6: 0.9840425531914895
layer 7: 0.9840425531914895
layer 8: 0.9840425531914895
Epoch: 82 | Batch: 000 / 037 | Total loss: 1.613 | Reg loss: 0.037 | Tree loss: 1.613 | Accuracy: 0.525391 | 0.794 sec/iter
Average sparseness: 0.9840425531914894
layer 0

...

Epoch: 300 | Batch: 000 / 037 | Total loss: 1.602 | Reg loss: 0.039 | Tree loss: 1.602 | Accuracy: 0.556641 | 0.786 sec/iter
Average sparseness: 0.9840425531914894
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
layer 5: 0.9840425531914894
layer 6: 0.9840425531914895
layer 7: 0.9840425531914895
layer 8: 0.9840425531914895
Epoch: 301 | Batch: 000 / 037 | Total loss: 1.646 | Reg loss: 0.039 | Tree loss: 1.646 | Accuracy: 0.519531 | 0.786 sec/iter
Average sparseness: 0.9840425531914894
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
layer 5: 0.9840425531914894
layer 6: 0.9840425531914895
layer 7: 0.9840425531914895
layer 8: 0.9840425531914895
Epoch: 302 | Batch: 000 / 037 | Total loss: 1.633 | Reg loss: 0.039 | Tree loss: 1.633 | Accuracy: 0.507812 | 0.786 sec/iter
Average sparseness: 0.9840425531914894
laye

Epoch: 320 | Batch: 000 / 037 | Total loss: 1.646 | Reg loss: 0.039 | Tree loss: 1.646 | Accuracy: 0.527344 | 0.786 sec/iter
Average sparseness: 0.9840425531914894
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
layer 5: 0.9840425531914894
layer 6: 0.9840425531914895
layer 7: 0.9840425531914895
layer 8: 0.9840425531914895
Epoch: 321 | Batch: 000 / 037 | Total loss: 1.636 | Reg loss: 0.039 | Tree loss: 1.636 | Accuracy: 0.496094 | 0.786 sec/iter
Average sparseness: 0.9840425531914894
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
layer 5: 0.9840425531914894
layer 6: 0.9840425531914895
layer 7: 0.9840425531914895
layer 8: 0.9840425531914895
Epoch: 322 | Batch: 000 / 037 | Total loss: 1.605 | Reg loss: 0.039 | Tree loss: 1.605 | Accuracy: 0.521484 | 0.786 sec/iter
Average sparseness: 0.9840425531914894
laye

Epoch: 340 | Batch: 000 / 037 | Total loss: 1.626 | Reg loss: 0.039 | Tree loss: 1.626 | Accuracy: 0.484375 | 0.786 sec/iter
Average sparseness: 0.9840425531914894
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
layer 5: 0.9840425531914894
layer 6: 0.9840425531914895
layer 7: 0.9840425531914895
layer 8: 0.9840425531914895
Epoch: 341 | Batch: 000 / 037 | Total loss: 1.634 | Reg loss: 0.039 | Tree loss: 1.634 | Accuracy: 0.492188 | 0.786 sec/iter
Average sparseness: 0.9840425531914894
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
layer 5: 0.9840425531914894
layer 6: 0.9840425531914895
layer 7: 0.9840425531914895
layer 8: 0.9840425531914895
Epoch: 342 | Batch: 000 / 037 | Total loss: 1.658 | Reg loss: 0.039 | Tree loss: 1.658 | Accuracy: 0.472656 | 0.786 sec/iter
Average sparseness: 0.9840425531914894
laye

Epoch: 360 | Batch: 000 / 037 | Total loss: 1.608 | Reg loss: 0.039 | Tree loss: 1.608 | Accuracy: 0.511719 | 0.787 sec/iter
Average sparseness: 0.9840425531914894
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
layer 5: 0.9840425531914894
layer 6: 0.9840425531914895
layer 7: 0.9840425531914895
layer 8: 0.9840425531914895
Epoch: 361 | Batch: 000 / 037 | Total loss: 1.594 | Reg loss: 0.039 | Tree loss: 1.594 | Accuracy: 0.537109 | 0.787 sec/iter
Average sparseness: 0.9840425531914894
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
layer 5: 0.9840425531914894
layer 6: 0.9840425531914895
layer 7: 0.9840425531914895
layer 8: 0.9840425531914895
Epoch: 362 | Batch: 000 / 037 | Total loss: 1.629 | Reg loss: 0.039 | Tree loss: 1.629 | Accuracy: 0.498047 | 0.787 sec/iter
Average sparseness: 0.9840425531914894
laye

Epoch: 380 | Batch: 000 / 037 | Total loss: 1.632 | Reg loss: 0.039 | Tree loss: 1.632 | Accuracy: 0.490234 | 0.787 sec/iter
Average sparseness: 0.9840425531914894
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
layer 5: 0.9840425531914894
layer 6: 0.9840425531914895
layer 7: 0.9840425531914895
layer 8: 0.9840425531914895
Epoch: 381 | Batch: 000 / 037 | Total loss: 1.637 | Reg loss: 0.039 | Tree loss: 1.637 | Accuracy: 0.472656 | 0.787 sec/iter
Average sparseness: 0.9840425531914894
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
layer 5: 0.9840425531914894
layer 6: 0.9840425531914895
layer 7: 0.9840425531914895
layer 8: 0.9840425531914895
Epoch: 382 | Batch: 000 / 037 | Total loss: 1.626 | Reg loss: 0.039 | Tree loss: 1.626 | Accuracy: 0.503906 | 0.787 sec/iter
Average sparseness: 0.9840425531914894
laye

In [23]:
plt.figure(figsize=(10, 5))
plt.ylabel("Accuracy")
plt.xlabel('Iteration')
plt.plot(accs, label='Accuracy vs iteration')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [24]:
plt.figure()
plt.ylabel("Loss")
plt.xlabel('Iteration')
plt.plot(losses, label='Loss vs iteration')
# plt.yscale("log")
plt.show()

plt.figure()
weights = tree.inner_nodes.weight.cpu().detach().numpy().flatten()
plt.hist(weights, bins=500)
weights_std = np.std(weights)
weights_mean = np.mean(weights)
plt.axvline(weights_mean + weights_std, color='r')
plt.axvline(weights_mean - weights_std, color='r')
plt.title(f"Mean: {weights_mean}   |   STD: {weights_std}")
plt.yscale("log")
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Tree Visualization

In [25]:
plt.figure(figsize=(15, 10), dpi=80)
avg_height, root = tree.visualize()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Average height: 7.51063829787234


# Extract Rules

# Accumulate samples in the leaves

In [26]:
print(f"Number of patterns: {len(root.get_leaves())}")

Number of patterns: 47


In [27]:
method = 'greedy'

In [28]:
root.clear_leaves_samples()

with torch.no_grad():
    for batch_idx, (data, target) in enumerate(tree_loader):
        root.accumulate_samples(data, method)

/home/eitan.k/.local/lib/python3.6/site-packages/torch/nn/functional.py:1709: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


# Tighten boundaries

In [29]:
attr_names = [f"T_{i}" for i in range(test_samples.shape[2])]
leaves = root.get_leaves()
sum_comprehensibility = 0
comprehensibilities = []
for pattern_counter, leaf in enumerate(leaves):
    leaf.reset_path()
    leaf.tighten_with_accumulated_samples()
    conds = leaf.get_path_conditions(attr_names)
    print(f"============== Pattern {pattern_counter + 1} ==============")
    comprehensibilities.append(sum([cond.comprehensibility for cond in conds]))
    
print(f"Average comprehensibility: {np.mean(comprehensibilities)}")
print(f"std comprehensibility: {np.std(comprehensibilities)}")
print(f"var comprehensibility: {np.var(comprehensibilities)}")
print(f"minimum comprehensibility: {np.min(comprehensibilities)}")
print(f"maximum comprehensibility: {np.max(comprehensibilities)}")

============== Pattern 1 ==============
============== Pattern 2 ==============
8
============== Pattern 3 ==============
57
============== Pattern 4 ==============
============== Pattern 5 ==============
============== Pattern 6 ==============
============== Pattern 7 ==============
39
============== Pattern 8 ==============
============== Pattern 9 ==============
============== Pattern 10 ==============
============== Pattern 11 ==============
============== Pattern 12 ==============
============== Pattern 13 ==============
============== Pattern 14 ==============
============== Pattern 15 ==============
============== Pattern 16 ==============
5190
============== Pattern 17 ==============
============== Pattern 18 ==============
2
============== Pattern 19 ==============
12
============== Pattern 20 ==============
7091
============== Pattern 21 ==============
============== Pattern 22 ==============
2952
============== Pattern 23 ==============
============== Pattern 24 ============

/home/eitan.k/EntangledExplainableClustering/soft_decision_tree/sdt_model.py:14: RuntimeWarning: divide by zero encountered in log
  return np.log(1 / (1 - x))
